In [1]:
# Import
import os,sys
CURRENT_TEST_DIR = os.getcwd()
sys.path.append(CURRENT_TEST_DIR + "/../new_iteration/")
import pickle
import argparse
from sklearn.metrics import confusion_matrix
from datetime import datetime

from vrae.vrae import VRAEC
from vrae.utils import *
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.optim as optim
from torch.utils import data as data2
from torch.utils.data.dataset import random_split
from torch.utils.data import DataLoader, TensorDataset
from datetime import datetime
from tas_utils_bs import get_trainValLoader, get_testLoader
import plotly

import matplotlib.pyplot as plt
from PIL import Image


In [2]:
# Set hyper params
num_class = 20
sequence_length_B = 400
sequence_length_I = 75
number_of_features_B = 19
number_of_features_I = 60
shuffle = True
hidden_size = 90
hidden_layer_depth = 1
latent_length = 40
batch_size = 32
learning_rate = 0.0005
n_epochs = 100
dropout_rate = 0.2
cuda = True # options: True, False
print_every=30
clip = True # options: True, False
max_grad_norm=5
header_B = None
header_I = "CNN"

w_r = 0.0001
w_k = 1
w_c = 1
np.random.seed(1)
torch.manual_seed(1)

### Load data and preprocess

In [3]:
# Load data
data_dir = '../../new_data_folder/'
kfold_number = 0

logDir = 'models_and_stat/'
# model_name_B = 'BT19_joint_hs_{}_ll_{}_hl{}_lr_{}_wr_{}_wk_{}_wc_{}_{}'.format(hidden_size, latent_length, hidden_layer_depth, learning_rate, w_r,w_k, w_c, str(kfold_number))
# model_name_I = 'IcubCNN_joint_hs_{}_ll_{}_hl{}_lr_{}_wr_{}_wk_{}_wc_{}_{}'.format(hidden_size, latent_length, hidden_layer_depth, learning_rate, w_r,w_k, w_c, str(kfold_number))
model_name_B = 'testB'
model_name_I = 'testI'
device = torch.device("cuda:1")

train_loader, val_loader, train_dataset, val_dataset = get_trainValLoader(data_dir, k=kfold_number, spike_ready=False, batch_size=batch_size, shuffle=False)
test_loader, test_dataset = get_testLoader(data_dir, spike_ready=False, batch_size=batch_size, shuffle=False)

In [4]:
# Load pretrained model
model_B_pretrained = VRAEC(num_class=num_class,
            sequence_length=sequence_length_B,
            number_of_features = number_of_features_B,
            hidden_size = hidden_size, 
            hidden_layer_depth = hidden_layer_depth,
            latent_length = latent_length,
            batch_size = batch_size,
            learning_rate = learning_rate,
            n_epochs = n_epochs,
            dropout_rate = dropout_rate, 
            cuda = cuda,
            print_every=print_every, 
            clip=clip, 
            max_grad_norm=max_grad_norm,
            dload = logDir,
            model_name=model_name_B,
            header=header_B,
            device = device)

model_B_pretrained.load_state_dict(torch.load(logDir + model_name_B + '.pt'))
model_B_pretrained.eval()

model_I_pretrained = VRAEC(num_class=num_class,
            sequence_length=sequence_length_I,
            number_of_features = number_of_features_I,
            hidden_size = hidden_size, 
            hidden_layer_depth = hidden_layer_depth,
            latent_length = latent_length,
            batch_size = batch_size,
            learning_rate = learning_rate,
            n_epochs = n_epochs,
            dropout_rate = dropout_rate, 
            cuda = cuda,
            print_every=print_every, 
            clip=clip, 
            max_grad_norm=max_grad_norm,
            dload = logDir,
            model_name=model_name_I,
            header=header_I,
            device = device)
model_I_pretrained.load_state_dict(torch.load(logDir + model_name_I + '.pt'))
model_I_pretrained.eval()

model_B_trained = VRAEC(num_class=num_class,
            sequence_length=sequence_length_B,
            number_of_features = number_of_features_B,
            hidden_size = hidden_size, 
            hidden_layer_depth = hidden_layer_depth,
            latent_length = latent_length,
            batch_size = batch_size,
            learning_rate = learning_rate,
            n_epochs = n_epochs,
            dropout_rate = dropout_rate, 
            cuda = cuda,
            print_every=print_every, 
            clip=clip, 
            max_grad_norm=max_grad_norm,
            dload = logDir,
            model_name=model_name_B,
            header=header_B,
            device = device)

model_I_trained = VRAEC(num_class=num_class,
            sequence_length=sequence_length_I,
            number_of_features = number_of_features_I,
            hidden_size = hidden_size, 
            hidden_layer_depth = hidden_layer_depth,
            latent_length = latent_length,
            batch_size = batch_size,
            learning_rate = learning_rate,
            n_epochs = n_epochs,
            dropout_rate = dropout_rate, 
            cuda = cuda,
            print_every=print_every, 
            clip=clip, 
            max_grad_norm=max_grad_norm,
            dload = logDir,
            model_name=model_name_I,
            header=header_I,
            device = device)

# print(model_B_trained.state_dict().keys())
# dict_keys(['encoder.lstm.weight_ih_l0', 'encoder.lstm.weight_hh_l0', 'encoder.lstm.bias_ih_l0', 'encoder.lstm.bias_hh_l0', 'decoder.model.weight_ih_l0', 'decoder.model.weight_hh_l0', 'decoder.model.bias_ih_l0', 'decoder.model.bias_hh_l0', 'decoder.latent_to_hidden.weight', 'decoder.latent_to_hidden.bias', 'decoder.hidden_to_output.weight', 'decoder.hidden_to_output.bias', 'lmbd.hidden_to_mean.weight', 'lmbd.hidden_to_mean.bias', 'lmbd.hidden_to_logvar.weight', 'lmbd.hidden_to_logvar.bias', 'classifier.0.weight', 'classifier.0.bias'])
# print(model_I_trained.state_dict().keys())
# keys(['encoder.cnn.seq.0.weight', 'encoder.cnn.seq.0.bias', 'encoder.lstm.weight_ih_l0', 'encoder.lstm.weight_hh_l0', 'encoder.lstm.bias_ih_l0', 'encoder.lstm.bias_hh_l0', 'decoder.dcnn.0.weight', 'decoder.dcnn.0.bias', 'decoder.model.weight_ih_l0', 'decoder.model.weight_hh_l0', 'decoder.model.bias_ih_l0', 'decoder.model.bias_hh_l0', 'decoder.latent_to_hidden.weight', 'decoder.latent_to_hidden.bias', 'decoder.hidden_to_output.weight', 'decoder.hidden_to_output.bias', 'lmbd.hidden_to_mean.weight', 'lmbd.hidden_to_mean.bias', 'lmbd.hidden_to_logvar.weight', 'lmbd.hidden_to_logvar.bias', 'classifier.0.weight', 'classifier.0.bias'])

# load origin pretrained_model
model_B_trained.load_state_dict(model_B_pretrained.state_dict())
model_I_trained.load_state_dict(model_I_pretrained.state_dict())

model_B_trained.to(device)
model_I_trained.to(device)
model_B_trained.eval()
model_I_trained.eval()

/home/students/student9_02/anaconda3/lib/python3.7/site-packages/torch/nn/modules/rnn.py:50: UserWarning:

dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1



VRAE(n_epochs=100,batch_size=32,cuda=True)

criterion = nn.CrossEntropyLoss()
optimB = optim.Adam(model_B.parameters(), lr=learning_rate)
optimI = optim.Adam(model_I.parameters(), lr=learning_rate)
cl_loss_fn = nn.NLLLoss()
recon_loss_fn = nn.MSELoss()

In [5]:
model_B_trained.is_fitted = True
model_I_trained.is_fitted = True

model_B_trained = VRAEC(num_class=num_class,
            sequence_length=sequence_length_B,
            number_of_features = number_of_features_B,
            hidden_size = hidden_size, 
            hidden_layer_depth = hidden_layer_depth,
            latent_length = latent_length,
            batch_size = batch_size,
            learning_rate = learning_rate,
            n_epochs = n_epochs,
            dropout_rate = dropout_rate, 
            cuda = cuda,
            print_every=print_every, 
            clip=clip, 
            max_grad_norm=max_grad_norm,
            dload = logDir,
            model_name=model_name_B,
            header=header_B,
            w_r = w_r, 
            w_k = w_k, 
            w_c = w_c,
            device = device)
model_B_trained.load_state_dict(torch.load(logDir + model_name_B + '.pt'))
model_B_trained.to(device)
model_B_trained.eval()

model_I_trained = VRAEC(num_class=num_class,
            sequence_length=sequence_length_I,
            number_of_features = number_of_features_I,
            hidden_size = hidden_size, 
            hidden_layer_depth = hidden_layer_depth,
            latent_length = latent_length,
            batch_size = batch_size,
            learning_rate = learning_rate,
            n_epochs = n_epochs,
            dropout_rate = dropout_rate, 
            cuda = cuda,
            print_every=print_every, 
            clip=clip, 
            max_grad_norm=max_grad_norm,
            dload = logDir,
            model_name=model_name_I,
            header=header_I,
            w_r = w_r, 
            w_k = w_k, 
            w_c = w_c,
            device = device)
model_I_trained.load_state_dict(torch.load(logDir + model_name_I + '.pt'))
model_I_trained.to(device)
model_I_trained.eval()

In [6]:
from PIL import Image
from IPython.display import Image
from matplotlib import cm

In [7]:
import numpy as np

def scale(X, x_min, x_max):
    nom = (X-X.min(axis=0))*(x_max-x_min)
    denom = X.max(axis=0) - X.min(axis=0)
    denom[denom==0] = 1
    return x_min + nom/denom 


In [8]:
# # Visualize feature maps of the decoder I output
# %matplotlib inline
# def get_activation(name):
#     def hook(model, input, output):
#         activation[name] = output.detach()
#     return hook

# activation = {}
# act_dict = {}
# decoded_dict = {}
# test_num = 0
# for i, (XI, XB,  y) in enumerate(train_loader):
#     print("processing {} batch".format(i))
#     XI, XB, y = XI.to(device), XB.to(device), y.long().to(device)

#     if XI.size()[0] != batch_size:
# #             print("batch {} size {} < {}, skip".format(i, x.size()[0], batch_size))
#         break

#     test_num += XI.size(0)

#     # test modelI 
#     x_decoded, latent, output = model_I_trained(XI)
    
    
#     x_decoded = x_decoded.data.cpu().numpy()
#     # create decoded dict based on class
#     for i in range(y.size(0)):
#         label = y[i].data.cpu().item()
#         if label not in decoded_dict:
#             decoded_dict[label] = []
#         decoded_dict[label].append(x_decoded[i])
        
# pickle.dump(decoded_dict, open("decoded_dict_0420.pkl", "wb"))


In [10]:
# Visualize feature maps of the decoder B output
%matplotlib inline
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

activation = {}
act_dict = {}
decoded_dict = {}
test_num = 0
for i, (XI, XB,  y) in enumerate(train_loader):
    print("processing {} batch".format(i))
    XI, XB, y = XI.to(device), XB.to(device), y.long().to(device)

    if XB.size()[0] != batch_size:
#             print("batch {} size {} < {}, skip".format(i, x.size()[0], batch_size))
        break

    test_num += XB.size(0)

    # test modelI 
    x_decoded, latent, output = model_B_trained(XB)
    x_decoded = x_decoded.data.cpu().numpy()
    # create decoded dict based on class
    for i in range(y.size(0)):
        label = y[i].data.cpu().item()
        if label not in decoded_dict:
            decoded_dict[label] = []
        decoded_dict[label].append(x_decoded[i])
        
pickle.dump(decoded_dict, open("decoded_dict_B_0421.pkl", "wb"))

processing 0 batch
processing 1 batch
processing 2 batch
processing 3 batch
processing 4 batch
processing 5 batch
processing 6 batch
processing 7 batch
processing 8 batch
processing 9 batch
processing 10 batch
processing 11 batch
processing 12 batch
processing 13 batch
processing 14 batch
processing 15 batch
processing 16 batch
processing 17 batch
processing 18 batch


In [ ]:
#     print(decoded_dict.keys())
        
#         # try to plot one frame of one sample
#         plt.imshow(x_decoded[2,:,:,60])

# comment on 4.20, try to save the whole decoded_dict
#     for i in range(20):
#         if i in decoded_dict:
#             print("class {} has {} test samples".format(i, len(decoded_dict[i])))
            
            
#     # map the class_number to number of samples
#     def count_dict_class(key):
#         return [key, len(decoded_dict[key])]
  

#     pairs = map(count_dict_class, decoded_dict.keys())
#     pairs = np.array(list(pairs))
# #     print(pairs)
#     # find the class with maximum number of test samples
#     _, idx = np.argmax(pairs, axis=0)
    

#     class_number = pairs[idx][0]
#     num_samples = pairs[idx][1]
#     T = 75
#     print("max test samples, class {} has {} samples".format(class_number, num_samples))
    
#     # save all frames to make video
#     print("Saving dec_imgs...")
#     for n in range(num_samples):
#         myarray = decoded_dict[class_number][n][:, :, :]    
# #         print("myarray", myarray.shape) #(6, 10. 75)
#         arrayname = "Dec_imgs/Dec_C{}_S{}".format(class_number, n)
#         pickle.dump(myarray, open(arrayname+".pkl", 'wb'))
        
        
        
        
        
#         print("reload")
#         dec_array = pickle.load(open(arrayname+".pkl",'rb'))

#         for t in range(T):
#             fname = "Dec_imgs/Dec_C{}_S{}_T{}_c.png".format(class_number, n, t)
#             # normalize the data
#             data = myarray[:,:, t]
#             print(data.shape, type(data))
#             print(data)
#             norm_array = scale(data, 0, 1)
#             im = Image.fromarray((norm_array * 255).astype(np.uint8))
#             im = cm.gist_earth(im)
#             im = Image.fromarray(np.uint8(cm.gist_earth(norm_array)*255))
#             width, height = im.size
# #             print("width {}, height {}".format(width, height)) # 10, 6
#             newsize = (100, 60) 
#             im = im.resize(newsize) 

        
#             im.save(fname)
#             display(im)
#             raise ValueError("check check")

           
                
                


#     '''
#     Plot the class with maximum number of samples to visualize similarity
#     '''
#     # Create subplot matrix
#     class_number = pairs[idx][0]
#     num_col = min(pairs[idx][1], 4)
#     # row for frames
#     num_row = 25
#     assert num_col > 1, "max number of samples {} <=1".format(num_col)
    
#     # create labels
#     col_labels = list(np.repeat(class_number, num_col))
#     cols = [str(ele) for ele in col_labels]

#     rows = ['CH {} '.format(row+1) for row in range(num_row)]
#     print("cols", cols, "rows", rows)

#     fig, axes = plt.subplots(nrows=len(rows), ncols=len(cols), figsize=(20,10)) 
    
#     # set labels
#     for ax, col in zip(axes[0], cols):
#         ax.set_title(col)

#     for ax, row in zip(axes[:,0], rows):
#         ax.set_ylabel(row, rotation=0, size='large')
    
#     # plot the figures
#     for col_idx in range(num_col):
#         for row_idx in range(num_row):
#             axes[row_idx, col_idx].imshow(decoded_dict[class_number][col_idx][:, :,row_idx])
    
#     fig.tight_layout()
    
#     plt.savefig("Dec_imgs/C{}_NS{}_NT{}.png".format(class_number, num_col, num_row))
#     plt.show()
    
    
    
#     '''
#     Plot for different class, to visualize the differences
#     '''
#     # Create subplot matrix
#     num_col = 3
#     class_numbers = np.random.choice(list(decoded_dict.keys()), size=num_col, replace=False)
#     class_number = pairs[idx][0]
    
#     print("plot for classes: ", class_numbers)
    
#     # row for frames
#     num_row = 20
#     assert num_col > 1, "max number of samples {} <=1".format(num_col)
    
#     # create labels
#     col_labels = list(class_numbers)
#     cols = [str(ele) for ele in col_labels]

#     rows = ['CH {} '.format(row+1) for row in range(num_row)]
#     print("cols", cols, "rows", rows)

#     fig, axes = plt.subplots(nrows=len(rows), ncols=len(cols), figsize=(20,30)) 
    
#     # set labels
#     for ax, col in zip(axes[0], cols):
#         ax.set_title(col)

#     for ax, row in zip(axes[:,0], rows):
#         ax.set_ylabel(row, rotation=0, size='large')
    
#     # plot the figures
#     for col_idx in range(num_col):
#         for row_idx in range(num_row):
#             # plot for the first sample of the specified class
#             axes[row_idx, col_idx].imshow(decoded_dict[class_numbers[col_idx]][0][:, :,row_idx])
    
#     fig.tight_layout()
#     plt.show()
    
        
        
    
# #     print("x_decoded", x_decoded.size()) # [32, 6, 10, 75]
# #     print(output.shape) # [32, 20]
    
    
# #     act_layer = 'dec_conv1' # output only one frame generated by latent_to_hidden
    
# #     act_layer = "decoder"

# #     model_I_trained.encoder.cnn.seq[0].register_forward_hook(get_activation(act_layer))

#     # output only one frame generated by latent_to_hidden
# #     model_I_trained.decoder.dcnn[0].register_forward_hook(get_activation(act_layer))
# #     model_I_trained.decoder.register_forward_hook(get_activation(act_layer))

    
#     act = activation[act_layer].squeeze()
#     print("act ", act.size()) # [32, 6, 10]
#     act = act.cpu()
    
#     # fill in the act_dict by class
#     for i in range(y.size(0)):
#         label = y[i].data.cpu().item()
#         if label not in act_dict:
#             act_dict[label] = []
#         act_dict[label].append(act[i])
#     print(act_dict.keys())
#     for i in range(20):
#         if i in act_dict:
#             print("class {} has {} test samples".format(i, len(act_dict[i])))



            
       
#     # Create subplot matrix
#     class_number = 5
#     num_col = len(act_dict[class_number])
#     if len(list(act.size()))==4:
#         num_row = act.size(3)
#     else:
#         num_row =1
    
#     # create labels
#     col_labels = list(np.repeat(class_number, num_col))
#     cols = [str(ele) for ele in col_labels]
    
# #     cols = [str(label) for label in y[:num_col].data.cpu().numpy()]
#     rows = ['CH {} of act_layer'.format(row+1) for row in range(num_row)]
#     print("cols", cols, "rows", rows)

# #     fig, axes = plt.subplots(nrows=len(rows), ncols=len(cols), figsize=(20, 15)) 

# #     for ax, col in zip(axes[0], cols):
# #         ax.set_title(col)

# #     for ax, row in zip(axes[:,0], rows):
# #         ax.set_ylabel(row, rotation=0, size='large')
        
# #     fig, axarr = plt.subplots(act.size(3))
    
# #     for col_idx in range(num_col):
# #         for row_idx in range(num_row):
# #             axes[row_idx, col_idx].imshow(act_dict[class_number][col_idx][:, :,row_idx])
#     fig, axes = plt.subplots(len(cols), figsize=(20, 15)) 
#     print("axes", axes)

# #     if num_col == 1:
# #         axes.set_title(cols[0])
# #     else:
# #         for col in cols:
# #             axes.set_title(col)

# #     axes.set_ylabel(row, rotation=0, size='large')
        
# #     fig, axarr = plt.subplots(act.size(3))
    
# #     for col_idx in range(num_col):
# #         for row_idx in range(num_row):
# #             axes[row_idx, col_idx].imshow(act_dict[class_number][col_idx][:, :,row_idx])
#     for col_idx in range(num_col):
#         axes[col_idx].imshow(act_dict[class_number][col_idx])
    

#     fig.tight_layout()
#     plt.show()
